# Jupyter Notebook to recreate the figures on the website

### Usage: Copy the notebook and the folder *webinput* into the *multiImage_pytorch* directory. Outputs are written to *./tmp/website*

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import numpy as np
from pathlib import Path
import subprocess
import sys
import torch

module_path = Path("./../multiImage_pytorch").absolute()
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))
    
from dataset import SvbrdfDataset
from utils import (enable_deterministic_random_engine, gamma_encode, 
                   read_image_tensor, write_image_tensor, write_image, 
                   unpack_svbrdf, pack_svbrdf, 
                   encode_as_unit_interval)

In [7]:
def render_animated(output_path, renderer, svbrdf, flip_cam=True):
    output_path.mkdir(parents=True, exist_ok=True)
    
    for i, phi in enumerate(np.linspace(0.0, np.pi*2, 120, endpoint=False)):
        cam       = Camera([0, 1, 1.4]) if flip_cam else Camera([0, -1, 1.4])
        light     = Light([1.7 * np.cos(phi), 1.7 * np.sin(phi), 1.7], [30, 30, 30]) if flip_cam else Light([-1.7 * np.cos(phi), -1.7 * np.sin(phi), 1.7], [30, 30, 30])
        scene     = Scene(cam, light)
        rendering = renderer.render(scene, svbrdf).squeeze(0)
        
        mapping   = OrthoToPerspectiveMapping(cam, (512, 512))
        rendering = mapping.apply(gamma_encode(rendering.clone().cpu().detach().permute(1, 2, 0)).numpy())

        output_file_path = output_path.joinpath("{:d}.png".format(i))
        write_image(output_file_path, rendering)
        
def make_mat_sample(svbrdf, image, spacer_width=10, return_maps=False):
    normals, diffuse, roughness, specular = unpack_svbrdf(svbrdf)
    normals = encode_as_unit_interval(normals)
    diffuse = gamma_encode(diffuse)
    specular = gamma_encode(specular)
    
    spacer = torch.ones((3, normals.shape[-2], spacer_width))
    
    mat_sample = torch.cat([normals, spacer, diffuse, spacer, roughness, spacer, specular, spacer, spacer, spacer, spacer, gamma_encode(image)], dim=-1)
    
    if return_maps:
        return mat_sample, normals, diffuse, roughness, specular
    else:
        return mat_sample
    
def make_video(input_path, output_path):
    # Create a video from sequence of images
    cmd = ["ffmpeg", "-y", "-r", "30", "-i", str(input_path / "%d.png"), "-pix_fmt", "yuv420p", str(output_path)]
    subprocess.run(cmd, check=True)

In [9]:
website_path = Path("./../../documentation/website")
data_path = Path("./data/website")
tmp_path = Path("./tmp")

### Loading SVBRDFs from a dataset

In this experiment, we load SVBRDFs from a dataset on disk and rely on the generation of artificial input images.

In [5]:
enable_deterministic_random_engine(131)

data = SvbrdfDataset(data_directory=str(data_path / "materials"), image_size=512, input_image_count=0, used_input_image_count=1, use_augmentation=True, scale_mode='crop')

for i, d in enumerate(data):
    mat_sample, normals, diffuse, roughness, specular = make_mat_sample(d['svbrdf'], d['inputs'][0], spacer_width=10, return_maps=True)
    
    write_image_tensor(website_path / "images/mat{:d}.png".format(i + 1), mat_sample)
    
    # Only write the individual maps for the first SVBRDF
    if i == 0:
        write_image_tensor(website_path / "images/normals{:d}.png".format(i + 1), normals)
        write_image_tensor(website_path / "images/diffuse{:d}.png".format(i + 1), diffuse)
        write_image_tensor(website_path / "images/roughness{:d}.png".format(i + 1), roughness)
        write_image_tensor(website_path / "images/specular{:d}.png".format(i + 1), specular)

### Material Mixing

We pick two materials from the given dataset and mix them together according to the algorithm given in the paper. This example of mixing stone with metal demonstrates, that not all combinations produce plausible real world materials.

In [13]:
enable_deterministic_random_engine(113)

data = SvbrdfDataset(data_directory=str(data_path / "materials"), image_size=512, input_image_count=0, used_input_image_count=1, use_augmentation=True, scale_mode='crop')

_, svbrdf1 = data.read_sample(data.file_paths[0])
_, svbrdf2 = data.read_sample(data.file_paths[2])
svbrdf1x2 = data.mix(svbrdf1, svbrdf2, 0.3)

rendering = data.render_inputs(svbrdf1x2, 1)[0]

write_image_tensor(website_path / "images/mat1xmat3.png", make_mat_sample(svbrdf1x2, rendering))

### Missing Ambient Lighting

This experiment demonstrates that the absence of ambient lighting during input image generation can result in images with reduced information content.

In [15]:
enable_deterministic_random_engine(313)

data = SvbrdfDataset(data_directory=str(data_path / "metal"), image_size=512, input_image_count=0, used_input_image_count=1, use_augmentation=True, scale_mode='crop')
write_image_tensor(website_path / "images/metal_wo_ambient.png", gamma_encode(data[0]['inputs'][0]))

### L1 Loss vs Rendering Loss

Here, we just reorganize images from a figure in the paper that demonstrates the effects of using only L1 loss between the material maps versus using the rendering loss.

In [16]:
enable_deterministic_random_engine(313)

in_directory  = data_path / "l1vsrendering"
out_directory = website_path / "images"
variants      = ["gt", "l1", "rendering"]

for variant in variants:
    normals   = read_image_tensor(str(in_directory / "{}_normals.png".format(variant)))
    diffuse   = read_image_tensor(str(in_directory / "{}_diffuse.png".format(variant)))
    roughness = read_image_tensor(str(in_directory / "{}_roughness.png".format(variant)))
    specular  = read_image_tensor(str(in_directory / "{}_specular.png".format(variant)))
    rendering = read_image_tensor(str(in_directory / "{}_rendering.png".format(variant)))
    
    # Spacer has half the size since these images are 256x256 instead of 512x512
    spacer = torch.ones((3, normals.shape[-2], 5))
    
    mat_sample = torch.cat([normals, spacer, diffuse, spacer, roughness, spacer, specular, spacer, spacer, spacer, spacer, rendering], dim=-1)
    
    write_image_tensor(str(out_directory / "l1vsrendering_{}.png".format(variant)), mat_sample)

Visualization ortho to perspective mapping

In [10]:
from environment import Camera, Light, Scene
from renderers import LocalRenderer, OrthoToPerspectiveMapping
import pytweening as pt

data = SvbrdfDataset(data_directory=str(data_path / "materials"), image_size=512, input_image_count=0, used_input_image_count=1, use_augmentation=True, scale_mode='crop')

renderer = LocalRenderer()

# Original: 
# cam    = Camera([-0.5, -1.8, 1.2])             
# light  = Light([1, 1, 1.7], [30, 30, 30]) 
cam    = Camera([-0.5, -1.8, 0.2])
light  = Light([1, 1, 0.7], [30, 30, 30]) 
scene  = Scene(cam, light)

# Render
image = torch.clamp(renderer.render(scene, data[2]["svbrdf"])[0].cpu().permute(1, 2, 0), min=0.0, max=1.0)
image = gamma_encode(image).numpy()

import matplotlib.pyplot as plt
%matplotlib inline

mapping = OrthoToPerspectiveMapping(cam, (512, 512))

import numpy as np

outdir = tmp_path / "mapping"
outdir.mkdir(parents=True, exist_ok=True)
for i, t in enumerate(np.linspace(-1.0, 1.0, 120, endpoint=True)):
    outdir_static = outdir / "static"
    outdir_static.mkdir(parents=True, exist_ok=True)
    # Previous easeInOutCubic
    write_image(outdir_static / "{:d}.png".format(i), mapping.apply(image, pt.easeInOutQuint(np.abs(t))))
    
    phi = 2*np.pi*(t+1.0)*0.5
    light_anim = Light([light.pos[0] * np.cos(phi), light.pos[1] * np.sin(phi), light.pos[2]], light.color)
    scene_anim = Scene(cam, light_anim)
    image_anim = torch.clamp(renderer.render(scene_anim, data[2]["svbrdf"])[0].cpu().permute(1, 2, 0), min=0.0, max=1.0)
    image_anim = gamma_encode(image_anim).numpy()
    
    outdir_dynamic = outdir / "dynamic"
    outdir_dynamic.mkdir(parents=True, exist_ok=True)
    write_image(outdir_dynamic / "{:d}.png".format(i), mapping.apply(image_anim, pt.easeInOutQuint(np.abs(t))))
    
make_video(outdir_static, website_path / "videos" / "patchsampling_static_light.mp4")
make_video(outdir_dynamic, website_path / "videos" / "patchsampling_dynamic_light.mp4")

...

In [30]:
from IPython.display import clear_output
from renderers import RednerRenderer, LocalRenderer
from environment import Camera, Light, Scene

data = SvbrdfDataset(data_directory=R"./data/train", 
                     image_size=256, scale_mode='crop', input_image_count=10, used_input_image_count=0, 
                     use_augmentation=False, mix_materials=False,
                     no_svbrdf=False, is_linear=False)

# Define loss function
class FixedSceneLoss(torch.nn.Module):
    def __init__(self, renderer):
        super(FixedSceneLoss, self).__init__()
        
        self.renderer = renderer
        
        cam      = Camera([0, -1, 1.4])             # This configuration breaks the fixed evaluation!
        light    = Light([1, 1, 1.7], [30, 30, 30]) #
        #cam      = Camera([0, 0, 2])
        #light    = Light([1, 1, 0.5], [20, 20, 20])
        self.scene  = Scene(cam, light)
        
        self.target_rendering   = None
        
    def forward(self, input, target):
        if self.target_rendering is None:
            self.target_rendering = self.renderer.render(self.scene, target)

        estimated_rendering = self.renderer.render(self.scene, input)
        
        loss = torch.nn.functional.l1_loss(self.target_rendering, estimated_rendering)
        
        # Be conformant to the rendering loss and return "multiple" renderings for
        # each batch instance 
        return loss, estimated_rendering.unsqueeze(0), self.target_rendering.unsqueeze(0)
        

output_dir             = Path("./tmp")
output_dir_optim_fixed = output_dir / "optim_fixed"
output_dir_optim_fixed.mkdir(parents=True, exist_ok=True)
output_dir_optim_fixed_svbrdf = output_dir_optim_fixed / "svbrdf"
output_dir_optim_fixed_svbrdf.mkdir(parents=True, exist_ok=True)

# Setup the loss function
from losses import RenderingLoss
renderer = LocalRenderer()
#renderer = RednerRenderer()
loss_function = RenderingLoss(renderer)
#loss_function = FixedSceneLoss(renderer)

svbrdf_t           = data[1]['svbrdf']
n_t, d_t, r_t, s_t = unpack_svbrdf(svbrdf_t)

optimization = "normals"

# Estimated SVBRDF maps
# Initialization with rand_like is important for a strong initial guess!
n_e = torch.rand_like(n_t, requires_grad=True) if optimization == "normals" else n_t.clone()
d_e = torch.rand_like(d_t, requires_grad=True) if optimization == "diffuse" else d_t.clone()
r_e = torch.rand(r_t.shape[-2:], requires_grad=True) if optimization == "roughness" else r_t.clone()
s_e = torch.rand_like(s_t, requires_grad=True) if optimization == "specular" else s_t.clone()

params = []
lr     = 5e-2
if optimization == "normals":
    params.append(n_e)
    lr = 5e-2
elif optimization == "diffuse":
    params.append(d_e)
    lr = 6e-3
elif optimization == "roughness":
    params.append(r_e)
    lr = 6e-3
elif optimization == "specular":    
    params.append(s_e)
    lr = 6e-3
    
optimizer = torch.optim.Adam(params, lr=lr)

losses  = []
svbrdfs = []
renderings_e = []
renderings_t = []
for i in range(200):
    optimizer.zero_grad()

    svbrdf_e = None
    if optimization == "normals":
        n_e_upperhemi  = torch.cat([n_e[:2], torch.clamp(n_e[2:], min=0.0001)])
        n_e_normalized = n_e_upperhemi / (torch.sqrt(torch.sum(n_e_upperhemi**2, dim=0, keepdim=True)) + 0.001) #torch.nn.functional.normalize(n_e_upperhemi, dim=0)
        svbrdf_e = pack_svbrdf(n_e_normalized, d_e, r_e, s_e)
    elif optimization == "diffuse":
        d_e_clamped = torch.clamp(d_e, min=0.001, max=1.0)
        svbrdf_e = pack_svbrdf(n_e, d_e_clamped, r_e, s_e)
    elif optimization == "roughness":
        r_e_clamped = torch.clamp(r_e, min=0.001, max=1.0)
        svbrdf_e = pack_svbrdf(n_e, d_e, torch.stack([r_e_clamped, r_e_clamped, r_e_clamped]), s_e)
    elif optimization == "specular":
        s_e_clamped = torch.clamp(s_e, min=0.001, max=1.0)
        svbrdf_e = pack_svbrdf(n_e, d_e, r_e, s_e_clamped)
        
    loss, rendering_e, rendering_t = loss_function(svbrdf_e.unsqueeze(0), svbrdf_t.unsqueeze(0))
    renderings_e.append(rendering_e[0].cpu().clone().detach())
    renderings_t.append(rendering_t[0].cpu().clone().detach())
    loss.backward()    
    optimizer.step()
    
    clear_output(wait=True)
    svbrdfs.append(svbrdf_e.cpu().clone().detach())
    losses.append(loss.item())
    print("({:03d}) Loss: {:f}".format(i, loss.item()))

(199) Loss: 0.049428


In [31]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import FormatStrFormatter

output_dir_optim_fixed_plots = output_dir_optim_fixed / "plots"
output_dir_optim_fixed_plots.mkdir(parents=True, exist_ok=True)

plt.ioff()

for i in range(len(losses)):
    clear_output(wait=True)
    dpi = mpl.rcParams['figure.dpi']
    fig = plt.figure(figsize=(1620/dpi, 260/dpi))

    ax = plt.subplot(1, 3, 1)
    #ax.yaxis.set_major_formatter(FormatStrFormatter('%0.01f'))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    ax.set_ylim(bottom=0.0, top=max(losses))
    ax.set_xlim(left=0, right=i+1)
    ax.xaxis.set_tick_params(labelsize=20)
    ax.yaxis.set_tick_params(labelsize=20)
    plt.plot(losses[:i])
    
    plt.subplot(1, 6, 3)
    n_t, d_t, r_t, s_t = unpack_svbrdf(svbrdf_t)
    if optimization == "normals":
        plt.imshow(encode_as_unit_interval(n_t).permute(1, 2, 0).numpy())
    elif optimization == "diffuse":
        plt.imshow(gamma_encode(d_t).permute(1, 2, 0).numpy())
    elif optimization == "roughness":
        plt.imshow(r_t.permute(1, 2, 0).numpy())
    elif optimization == "specular":
        plt.imshow(gamma_encode(s_t).permute(1, 2, 0).numpy())        
    plt.axis('off')
    plt.subplot(1, 6, 4)
    plt.imshow((gamma_encode(torch.mean(renderings_t[i], dim=0)).detach().cpu().permute(1, 2, 0).numpy()))
    plt.axis('off')
    plt.subplot(1, 6, 5)
    n_e, d_e, r_e, s_e = unpack_svbrdf(svbrdfs[i])
    if optimization == "normals":
        plt.imshow(encode_as_unit_interval(n_e).permute(1, 2, 0).numpy())
    elif optimization == "diffuse":
        plt.imshow(gamma_encode(d_e).permute(1, 2, 0).numpy())
    elif optimization == "roughness":
        plt.imshow(r_e.permute(1, 2, 0).numpy())
    elif optimization == "specular":
        plt.imshow(gamma_encode(s_e).permute(1, 2, 0).numpy())                
    plt.axis('off')
    plt.subplot(1, 6, 6)
    plt.imshow((gamma_encode(torch.mean(renderings_e[i], dim=0)).detach().cpu().permute(1, 2, 0).numpy()))
    plt.axis('off')
    
    plt.savefig(str(output_dir_optim_fixed_plots.joinpath("{:d}.png".format(i))), bbox_inches='tight')
    
    plt.close(fig)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [3]:
from environment import Camera, Light, Scene
from renderers import LocalRenderer, RednerRenderer, OrthoToPerspectiveMapping
import numpy as np
import pyredner


        
data = SvbrdfDataset(data_directory="./../../documentation/website/images/raw/materials", image_size=512, input_image_count=0, used_input_image_count=1, use_augmentation=True, scale_mode='crop')

outdir = Path("tmp/rendering")
outdir.mkdir(parents=True, exist_ok=True)

render_animated(outdir / "local", LocalRenderer(), data[2]["svbrdf"])
render_animated(outdir / "pathtracing", RednerRenderer(), data[2]["svbrdf"])

Using device 'cuda' and camera type 'CameraType.fullpatchsample' for redner
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([3,

## Render

In [57]:
origins = ["artificial", "real"]
methods = ["reference", "local", "pathtracing"]

renderer = LocalRenderer()

for origin in origins:
    origin_dir = Path("./../../documentation/website/data/estimation") / origin
    
    image_dataset = SvbrdfDataset(data_directory=origin_dir, image_size=256, input_image_count=1, used_input_image_count=1, no_svbrdf=True, is_linear=True, use_augmentation=True, scale_mode='crop')
    
    for i, image_data in enumerate(image_dataset):
        output_dir = Path("./tmp/inputs") / origin
        output_dir.mkdir(parents=True, exist_ok=True)
        write_image_tensor(output_dir / "{}.png".format(i), gamma_encode(image_data["inputs"][0]))
    
    for method in methods:
        method_dir = origin_dir / method
        dataset = SvbrdfDataset(data_directory=method_dir, image_size=256, input_image_count=0, used_input_image_count=0, use_augmentation=True, scale_mode='crop')
        
        method_out_dir = Path("./tmp/renderings") / origin / method
        
        for i, data in enumerate(dataset):
            output_dir = method_out_dir / "{}".format(i)
            output_dir.mkdir(parents=True, exist_ok=True)
            render_animated(output_dir, renderer, data["svbrdf"], flip_cam=False)
            
            # Create a video from 
            cmd = ["ffmpeg", "-y", "-r", "30", "-i", str(output_dir / "%d.png"), "-pix_fmt", "yuv420p", str(method_out_dir / "{}.mp4".format(i))]
            subprocess.run(cmd, check=True)

In [62]:
#methods = ["reference", "local", "pathtracing"]

data = SvbrdfDataset(data_directory="./../../documentation/website/data/estimation/artificial/pathtracing_broken_314", image_size=256, input_image_count=0, used_input_image_count=0, use_augmentation=True, scale_mode='crop')

for i, d in enumerate(data):
    normals, diffuse, roughness, specular = unpack_svbrdf(d['svbrdf'])
    normals  = encode_as_unit_interval(normals)
    diffuse  = gamma_encode(diffuse)
    specular = gamma_encode(specular)
    
    spacer = torch.ones((3, normals.shape[-2], 20))
    
    mat_sample = torch.cat([normals, spacer, diffuse, spacer, roughness, spacer, specular], dim=-1)
    write_image_tensor("./tmp/mat{:d}.png".format(i + 1), mat_sample)

In [55]:
from datetime import datetime, timedelta
import json

output_dir = Path("./tmp/losses")
output_dir.mkdir(parents=True, exist_ok=True)

variations = ["local", "pathtracing_broken_314"]

for variation in variations:
    # Read tensorboard export
    loss_path     = Path("./../../documentation/website/data/estimation") / "loss_{:s}.json".format(variation)
    segments_path = Path("./../../documentation/website/data/estimation") / "segments_{:s}.json".format(variation)

    loss = None
    with open(loss_path) as file:
        loss = json.load(file)

    t = [datetime.fromtimestamp(d[0]) for d in loss]
    x = [d[1] for d in loss]
    y = [d[2] for d in loss]

    mean_time_per_step = timedelta()
    for i in range(0, len(x) - 1):
        time_per_step = (t[i + 1] - t[i]) / (x[i + 1] - x[i])
        mean_time_per_step = mean_time_per_step + (time_per_step - mean_time_per_step) / (i + 1)
    
    print("mean time per mini batch: ", mean_time_per_step)
    print("mean time per mini batch: ", mean_time_per_step * 8)
    print("mean time per epoch: ", (1590*0.99) * mean_time_per_step)
    print("estimated training time: ", x[-1] * mean_time_per_step)
    
    dpi = mpl.rcParams['figure.dpi']
    fig = plt.figure(figsize=(640/dpi, 480/dpi))

    ax = plt.subplot(1, 1, 1)
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    ax.xaxis.set_tick_params(labelsize=20)
    ax.yaxis.set_tick_params(labelsize=20)
    plt.plot(x, y)
    plt.xlabel("Step", family='sans-serif', size=20)
    plt.ylabel("Loss", family='sans-serif', size=20)
    
    plt.savefig(str(Path("./tmp/losses") / "loss_{:s}.png".format(variation)), bbox_inches='tight')
    
    plt.close(fig)

mean time per mini batch:  0:00:02.469360
mean time per mini batch:  0:00:19.754880
mean time per epoch:  1:04:47.019576
estimated training time:  11 days, 13:00:48.566640
mean time per mini batch:  0:00:19.503548
mean time per mini batch:  0:02:36.028384
mean time per epoch:  8:31:40.534907
estimated training time:  20 days, 18:54:41.735320
